In [ ]:
%load_ext autoreload
%autoreload 2
CUDA_LAUNCH_BLOCKING=1
!nvidia-smi

In [ ]:
from __future__ import division, print_function
from typing import Dict, SupportsRound, Tuple, Any
from os import PathLike
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch,gc
from torch.autograd import grad
from torch.autograd import Variable
import torch.fft ############### Pytorch >= 1.8.0
import torch.nn.functional as F
import SimpleITK as sitk
import os, glob
import json
import subprocess
import sys
from PIL import Image
import torch.nn.functional as nnf
from torch.optim.lr_scheduler import CosineAnnealingLR,CosineAnnealingWarmRestarts,StepLR
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import torch
from torchvision import datasets, transforms
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
import random
import yaml
from tqdm import tqdm, trange
from numpy import zeros, newaxis
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
import json
import pickle
import cv2
import lagomorph as lm

import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

In [ ]:
################Parameter Loading#######################
def read_yaml(path):
    try:
        with open(path, 'r') as f:
            file = edict(yaml.load(f, Loader=yaml.FullLoader))
        return file
    except:
        print('NO FILE READ!')
        return None
    
para = read_yaml('./parameters.yml')

xDim = para.data.x 
yDim = para.data.y
zDim = para.data.z

def loss_Reg(y_pred):
        # For 3D reg
        # dy = torch.abs(y_pred[:, :, 1:, :, :] - y_pred[:, :, :-1, :, :])
        # dx = torch.abs(y_pred[:, :, :, 1:, :] - y_pred[:, :, :, :-1, :])
        # dz = torch.abs(y_pred[:, :, :, :, 1:] - y_pred[:, :, :, :, :-1])
        # dy = dy * dy
        # dx = dx * dx
        # dz = dz * dz
        # d = torch.mean(dx) + torch.mean(dy) + torch.mean(dz)
        # grad = d / 3.0

        dy = torch.abs(y_pred[:, :, 1:, :] - y_pred[:, :, :-1, :])
        dx = torch.abs(y_pred[:, :, :, 1:] - y_pred[:, :, :, :-1])

        dy = dy * dy
        dx = dx * dx
        d = torch.mean(dx) + torch.mean(dy) 
        grad = d / 2.0
        return grad

In [ ]:
### Load the dataset

# Load the NumPy array from the pickle file
with open('./datasets/mnist_10p_train_x.pkl', 'rb') as file:
    final_X_train = pickle.load(file)

with open('./datasets/mnist_10p_train_y.pkl', 'rb') as file:
    final_y_train = pickle.load(file)

with open('./datasets/mnist_10p_test_x.pkl', 'rb') as file:
    final_X_test = pickle.load(file)

with open('./datasets/mnist_10p_test_y.pkl', 'rb') as file:
    final_y_test = pickle.load(file)

In [ ]:
# ##################Training Data Loading##########################
# readfilename = './2DShape/train_data' + '.json'
# datapath = './2DShape/'
# data = json.load(open(readfilename, 'r'))
# outputs = []
# keyword = 'train'
# # outputs = np.array(outputs)

# for i in trange (0,len(data[keyword])):
#     filename_src = datapath + data[keyword][i]['source']
#     itkimage_src = sitk.ReadImage(filename_src)
#     source_scan = sitk.GetArrayFromImage(itkimage_src)
#     source_scan = cv2.resize(source_scan, (64, 64))
#     if source_scan.ndim == 3 and source_scan.shape == (64, 64, 4):
#         source_scan = source_scan[:,:,0]
    
#     filename_tar = datapath + data[keyword][i]['target']
#     itkimage_tar = sitk.ReadImage(filename_tar)
#     target_scan = sitk.GetArrayFromImage(itkimage_tar)
#     target_scan = cv2.resize(target_scan, (64, 64))
#     if target_scan.ndim == 3 and target_scan.shape == (64, 64, 4):
#         target_scan = target_scan[:,:,0]
        
#     # print(source_scan.shape, target_scan.shape)
    
#     # print(i, source_scan.min(), source_scan.max(), target_scan.min(), target_scan.max())
    
#     source_scan = (source_scan - np.min(source_scan)) / (np.max(source_scan) - np.min(source_scan))
#     target_scan = (target_scan - np.min(target_scan)) / (np.max(target_scan) - np.min(target_scan))
    
#     pair = np.concatenate((source_scan[newaxis,:], target_scan[newaxis,:]), axis=0)
#     outputs.append(pair)
#     # print(pair.shape)
#     # print(i, source_scan.min(), source_scan.max(), target_scan.min(), target_scan.max())

# train = torch.FloatTensor(outputs)
# print (train.shape)

# ##################Testing Data Loading##########################
# readfilename = './2DShape/test_data' + '.json'
# datapath = './2DShape/'
# data = json.load(open(readfilename, 'r'))
# outputs = []
# keyword = 'test'
# # outputs = np.array(outputs)

# for i in trange (0,len(data[keyword])):
#     filename_src = datapath + data[keyword][i]['source']
#     itkimage_src = sitk.ReadImage(filename_src)
#     source_scan = sitk.GetArrayFromImage(itkimage_src)
#     source_scan = cv2.resize(source_scan, (64, 64))
#     if source_scan.ndim == 3 and source_scan.shape == (64, 64, 4):
#         source_scan = source_scan[:,:,0]
    
#     filename_tar = datapath + data[keyword][i]['target']
#     itkimage_tar = sitk.ReadImage(filename_tar)
#     target_scan = sitk.GetArrayFromImage(itkimage_tar)
#     target_scan = cv2.resize(target_scan, (64, 64))
#     if target_scan.ndim == 3 and target_scan.shape == (64, 64, 4):
#         target_scan = target_scan[:,:,0]
        
#     # print(source_scan.shape, target_scan.shape)
    
#     # print(i, source_scan.min(), source_scan.max(), target_scan.min(), target_scan.max())
    
#     source_scan = (source_scan - np.min(source_scan)) / (np.max(source_scan) - np.min(source_scan))
#     target_scan = (target_scan - np.min(target_scan)) / (np.max(target_scan) - np.min(target_scan))
    
#     pair = np.concatenate((source_scan[newaxis,:], target_scan[newaxis,:]), axis=0)
#     outputs.append(pair)
#     # print(i, source_scan.min(), source_scan.max(), target_scan.min(), target_scan.max())

# test = torch.FloatTensor(outputs)#.unsqueeze(1)
# print (test.shape)

In [ ]:
#################Network optimization########################

'''Check initilization'''
from losses import MSE, Grad
from network_epdiff import Unet
from torch.utils.data import Dataset

net = Unet(
    in_shape = (xDim, yDim),
    infeats = 2,
    nb_features = [[16, 32, 32], [32, 32, 32, 16, 16]],
    nb_levels = None,
    max_pool = 2,
    feat_mult = 1,
    nb_conv_per_level = 1,
    half_res = False,
    skip_connection = True    
)


class TDataset(Dataset):
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data = self.data[index]
        label = self.labels[index] if self.labels is not None else None
        return data, label



train_set = TDataset(final_X_train, final_y_train)
trainloader = torch.utils.data.DataLoader(train_set, batch_size = para.solver.batch_size, shuffle=True, num_workers=1)

test_set = TDataset(final_X_test, final_y_test)
testloader = torch.utils.data.DataLoader(test_set, batch_size = para.solver.batch_size, shuffle=True, num_workers=1)

running_loss = 0 
running_loss_val = 0
template_loss = 0
printfreq = 1
sigma = 0.02
repara_trick = 0.0
loss_array = torch.FloatTensor(para.solver.epochs,1).fill_(0)
loss_array_val = torch.FloatTensor(para.solver.epochs,1).fill_(0)


if(para.model.loss == 'L2'):
    criterion = nn.MSELoss()
elif (para.model.loss == 'L1'):
    criterion = nn.L1Loss()
if(para.model.optimizer == 'Adam'):
    optimizer = optim.Adam(net.parameters(), lr= para.solver.lr)
elif (para.model.optimizer == 'SGD'):
    optimizer = optim.SGD(net.parameters(), lr= para.solver.lr, momentum=0.9)
if (para.model.scheduler == 'CosAn'):
    scheduler = CosineAnnealingLR(optimizer, T_max=len(trainloader), eta_min=0)

optimizer_template = optim.Adam(net.parameters(), lr= para.solver.lr)
scheduler_template = CosineAnnealingLR(optimizer_template, T_max=len(trainloader), eta_min=0)

In [ ]:
# ##################Training###################################
########### Only MGAug Training ###########

n_actual_epochs = 200
total_loss = []
sigma = 0.1
net = net.to(dev)

for epoch in range(n_actual_epochs): #min(para.solver.epochs, n_actual_epochs)
    total= 0; 
    total_val = 0; 
    total_template = 0; 
    mse_loss = 0
    total = 0
    reg_loss = 0
    vae_loss = 0
    latent_f = []
    net.train()
    # print('epoch:', epoch)
    all_labels = []
    for j, image_data in enumerate(trainloader):
        inputs, batch_labels = image_data
        inputs = inputs.to(dev)
        b, c, w, h = inputs.shape
        optimizer.zero_grad()
        src_bch = inputs[:,0,...].reshape(b,1,w,h)
        tar_bch = inputs[:,1,...].reshape(b,1,w,h)
        x = torch.cat([src_bch, tar_bch], dim=1).to(dev)
        pred = net(x) #, registration = True     
        # pred = net(src_bch, tar_bch, registration = True) #, registration = True     
        # loss = criterion(pred[0], tar_bch) 

        Sdef = pred[0]
        tar = tar_bch
        recon_loss = torch.nn.MSELoss()(tar, Sdef)
        regularization = (pred[2]*pred[3]).sum() / (tar.numel())
        loss_total = recon_loss/(sigma*sigma) + 0.01*regularization + 0.001*pred[5]

        loss_total.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss_total.item()
        total += running_loss
        running_loss = 0.0
        
        mse_loss += recon_loss.item()
        reg_loss += 0.01*regularization.item()
        vae_loss += (0.001*pred[5].item())
        
    total_loss.append(total)
    print ('epoch:', epoch, 'training loss:', total, 'mse_loss:', mse_loss, 'reg_loss:', reg_loss, 'kld_loss', vae_loss, end="\r") 
    
# torch.save(net, './saved_models/mgaug_lddmm_mnist_10p.pth')

In [ ]:
def set_random_seeds():
    seed = random.randint(1, 100)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # if you're using CUDA
    torch.cuda.manual_seed_all(seed)  # For multi-GPU

In [ ]:
# Define the model
class ConvNet(nn.Module):
    def __init__(self, num_classes=9):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=56, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=56, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Adjusted for input size 128x128
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape)
        x = x.view(-1, 64 * 16 * 16)  # Adjusted for input size 128x128
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# input_data = torch.randn(10, 1, 64, 64).to(dev)
# model = ConvNet(num_classes=10).to(dev)
# output = model(input_data)

In [ ]:
########### Classification Training ########### 

times = 3
num_classes = 10
epochs = 50
input_size = 64*64


final_acc = []
final_prec = []
final_rec = []
final_f1 = []

finalw_acc = []
finalw_prec = []
finalw_rec = []
finalw_f1 = []
tr_loss = []

restart = 5

for run in range(0,restart,1):
    set_random_seeds()
    
    model = ConvNet(num_classes=num_classes).to(dev)
    clf_optimizer = optim.Adam(model.parameters(), lr= 0.00001)
    criterion = nn.CrossEntropyLoss()

    aug_model = torch.load('./saved_models/mgaug_lddmm_mnist_10p.pth') 
    aug_model.eval()
    
    print("Run: ", run)
    for epoch in range(epochs):

        print('----------------')
        print('Epoch: ', epoch)
        print('----------------')

        total = 0
        acc = 0
        model.train()
        
        ##### train with ground truth images #####
        for j, image_data in enumerate(trainloader):
            inputs, batch_labels = image_data
            inputs = inputs.to(dev)
            b, c, w, h = inputs.shape

            src_bch = inputs[:,0,...].reshape(b,1,w,h)
            tar_bch = inputs[:,1,...].reshape(b,1,w,h)

            labels = [int(label) for label in batch_labels]
            labels_tensor = torch.tensor(labels, dtype=torch.long).to(dev)

            output = model(tar_bch)
            train_loss = criterion(output, labels_tensor)

            clf_optimizer.zero_grad()
            train_loss.backward()
            clf_optimizer.step()

            total += train_loss.item()
            
        ##### train with augmented images #####
        for time in range(times):
            for j, image_data in enumerate(trainloader):
                inputs, batch_labels = image_data
                inputs = inputs.to(dev)
                b, c, w, h = inputs.shape

                src_bch = inputs[:,0,...].reshape(b,1,w,h)
                tar_bch = inputs[:,1,...].reshape(b,1,w,h)

                with torch.no_grad():
                    x = torch.cat([src_bch, tar_bch], dim=1).to(dev)
                    pred = aug_model(x)
                    # pred = aug_model(src_bch, tar_bch, registration = True) 

                labels = [int(label) for label in batch_labels]
                labels_tensor = torch.tensor(labels, dtype=torch.long).to(dev)

                output = model(pred[0])
                train_loss = criterion(output, labels_tensor)

                clf_optimizer.zero_grad()
                train_loss.backward()
                clf_optimizer.step()

                total += train_loss.item()

        tr_loss.append(total)

        print ('epoch:', epoch, 'total training loss:', total)#, 'tr accuracy: ')#, acc/(len(trainloader)*times), end="\r") 

        ### without test-time augmentation ###
        print('------------------------------------------')
        print('Without Test Time Augmentation Performance')
        print('------------------------------------------')

        acc = 0
        predictions = []
        targets = []
        with torch.no_grad():
            model.eval()
            aug_model.eval()
            for j, image_data in enumerate(testloader):
                inputs, batch_labels = image_data
                inputs = inputs.to(dev)

                src_bch = inputs[:,0,...].reshape(b,1,w,h)
                tar_bch = inputs[:,1,...].reshape(b,1,w,h)

                outputs = model(tar_bch)
                _, predicted = torch.max(outputs, 1)

                labels = [int(label) for label in batch_labels]
                labels_tensor = torch.tensor(labels, dtype=torch.long).to(dev)

                predictions.extend(predicted.cpu().numpy())
                targets.extend(labels_tensor.cpu().numpy())

            overall_accuracyw = accuracy_score(targets, predictions)
            print("Testing Accuracy:", overall_accuracyw)

            precisionw = precision_score(targets, predictions, average='macro')
            print("Precision:", precisionw)

            recallw = recall_score(targets, predictions, average='macro')
            print("Recall:", recallw)

            f1w = f1_score(targets, predictions, average='macro')
            print("F1-score:", f1w)

        ### with test-time augmentation ###
        print('------------------------------------------')
        print('With Test Time Augmentation Performance')
        print('------------------------------------------')

        acc = 0
        predictions = []
        targets = []
        with torch.no_grad():
            model.eval()
            aug_model.eval()
            for j, image_data in enumerate(testloader):
                inputs, batch_labels = image_data
                inputs = inputs.to(dev)

                src_bch = inputs[:,0,...].reshape(b,1,w,h)
                tar_bch = inputs[:,1,...].reshape(b,1,w,h)

                outputs = model(tar_bch)
                _, predicted = torch.max(outputs, 1)

                labels = [int(label) for label in batch_labels]
                labels_tensor = torch.tensor(labels, dtype=torch.long).to(dev)

                correct = (predicted == labels_tensor).sum().item()
                accuracy = correct / labels_tensor.size(0) 

                predictions.extend(predicted.cpu().numpy())
                targets.extend(labels_tensor.cpu().numpy())

            for time in range(times):
                for j, image_data in enumerate(testloader):
                    inputs, batch_labels = image_data
                    inputs = inputs.to(dev)

                    src_bch = inputs[:,0,...].reshape(b,1,w,h)
                    tar_bch = inputs[:,1,...].reshape(b,1,w,h)

                    x = torch.cat([src_bch, tar_bch], dim=1).to(dev)
                    pred = aug_model(x) 

                    outputs = model(pred[0])
                    _, predicted = torch.max(outputs, 1)

                    labels = [int(label) for label in batch_labels]
                    labels_tensor = torch.tensor(labels, dtype=torch.long).to(dev)

                    correct = (predicted == labels_tensor).sum().item()
                    accuracy = correct / labels_tensor.size(0) 

                    predictions.extend(predicted.cpu().numpy())
                    targets.extend(labels_tensor.cpu().numpy())

            overall_accuracy = accuracy_score(targets, predictions)
            print("Testing Accuracy:", overall_accuracy)

            precision = precision_score(targets, predictions, average='macro')
            print("Precision:", precision)

            recall = recall_score(targets, predictions, average='macro')
            print("Recall:", recall)

            f1 = f1_score(targets, predictions, average='macro')
            print("F1-score:", f1)
           
    finalw_acc.append(overall_accuracyw)
    finalw_prec.append(precisionw)
    finalw_rec.append(recallw)
    finalw_f1.append(f1w)
        
    final_acc.append(overall_accuracy)
    final_prec.append(precision)
    final_rec.append(recall)
    final_f1.append(f1)

print("Model: Conv; Restarts: ", restart, "Times: ", times)
print('Acc with test-time aug: ', np.mean(final_acc), np.std(final_acc))
print('Acc without test-time aug: ', np.mean(finalw_acc), np.std(finalw_acc))
print('------------------------------------------------------')

print('Prec with test-time aug: ', np.mean(final_prec), np.std(final_prec))
print('Prec without test-time aug: ', np.mean(finalw_prec), np.std(finalw_prec))
print('------------------------------------------------------')

print('Rec with test-time aug: ', np.mean(final_rec), np.std(final_rec))
print('Rec without test-time aug: ', np.mean(finalw_rec), np.std(finalw_rec))
print('------------------------------------------------------')

print('F1-Sc with test-time aug: ', np.mean(final_f1), np.std(final_f1))
print('F1-Sc without test-time aug: ', np.mean(finalw_f1), np.std(finalw_f1))

torch.save(model, './saved_models/mgaug_lddmm_clf_mnist_10p_3t.pth')
torch.save(aug_model, './saved_models/mgaug_lddmm_mnist_10p.pth')